# Percepción (PER): examen de prácticas del bloque 1, grupo 3CO11, turno 2, 23-4-2024, 8:45-9:30

Lee este cuaderno y realiza las actividades y ejercicios propuestos.

## Importación de librerías relevantes

Ejecuta el código siguiente para importar librerías relevantes:

In [6]:
import warnings; warnings.filterwarnings("ignore")
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

## Lectura y partición del corpus default-of-credit-card-clients

Ejecuta el código siguiente con random_state igual a las tres últimas cifras de tu DNI/NIE:

In [2]:
X, y = fetch_openml("default-of-credit-card-clients", version=4, return_X_y=True, as_frame=False)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, shuffle=True, random_state=123)
N, D = X.shape; C = len(np.unique(y)); N_train = len(X_train); N_test = len(X_test)
print(f'N = {N}  D = {D}  C = {C}  N_train = {N_train}  N_test = {N_test}')

N = 13272  D = 21  C = 2  N_train = 11944  N_test = 1328


Tarea del [**tabular data learning benchmark**](https://arxiv.org/abs/2207.08815). Se trata de predecir si se va a producir impago o no (2 clases) con tarjetas de crédito a partir del crédito dado e información (financiera) del cliente.

## Experimento de referencia con LDA

Ejecuta el código siguiente para estudiar el error de LDA con ajuste de hiper-parámetros mediante GridSearchCV

In [3]:
%%timeit -n1 -r1
scaler = StandardScaler(); pca = PCA(); lda = LinearDiscriminantAnalysis()
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("lda", lda)])
G = {"pca__n_components": [.9, .99, None],
     "lda__tol": [1e-5, 1e-4, 1e-3]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
err = 1.0 - GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Error: {err:.1%} con {GS.best_params_}')

Error: 33.3% con {'lda__tol': 1e-05, 'pca__n_components': 0.99}
5.7 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


## Ejercicio 1

Replica el código del experimento de referencia y modifícalo para estudiar el error de QDA con ajuste de hiper-parámetros mediante GridSearchCV.

In [7]:
%%timeit -n1 -r1
scaler = StandardScaler(); pca = PCA(); qda = QuadraticDiscriminantAnalysis()
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("qda", qda)])
G = {"pca__n_components": [.9, .99, None],
     "qda__reg_param": [1e-2, .1, .5, .9, 1]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
err = 1.0 - GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Error: {err:.1%} con {GS.best_params_}')

Error: 32.7% con {'pca__n_components': 0.99, 'qda__reg_param': 1}
2.34 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


El error usando los mejores parámetros con QDA es menor que usando los mejores parámetros con LDA
Por lo tanto, sería mejor utilizar este modelo

## Ejercicio 2

Replica el código del experimento de referencia y modifícalo para estudiar el error de regresión logística con ajuste de hiper-parámetros mediante GridSearchCV. Incluye la posibilidad de preprocesar la entrada con características polinómicas seguida de estandarización y PCA.

In [8]:
%%timeit -n1 -r1
scaler = StandardScaler(); pca = PCA(); logreg = LogisticRegression()
pipe = Pipeline(steps=[("scaler", scaler), ("pca", pca), ("logreg", logreg)])
G = {"pca__n_components": [.9, .99, None],
     "logreg__solver": ["lbfgs", "liblinear"],
     "logreg__tol": [1e-5, 1e-4, 1e-3],
     "logreg__C": [.1, 1, 10]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
err = 1.0 - GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Error: {err:.1%} con {GS.best_params_}')

Error: 30.1% con {'logreg__C': 0.1, 'logreg__solver': 'liblinear', 'logreg__tol': 1e-05, 'pca__n_components': 0.99}
9.57 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


El error usando regresión logística con los mejores parámetros es menor que LDA y que QDA, por lo tanto es un modelo más preciso

Ahora comprobaremos si utilizar características polinómicas mejora la solución (la primera celda es equivalente al ejercicio anterior porque poly__degree = 1)

In [10]:
%%timeit -n1 -r1
scaler = StandardScaler(); pca = PCA(); logreg = LogisticRegression(); poly = PolynomialFeatures()
pipe = Pipeline(steps=[("scaler", scaler), ("poly", poly), ("pca", pca), ("logreg", logreg)])
G = {"pca__n_components": [.9, .99, None],
     "poly__degree": [1],
     "logreg__solver": ["lbfgs", "liblinear"],
     "logreg__tol": [1e-5, 1e-4, 1e-3],
     "logreg__C": [.1, 1, 10]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
err = 1.0 - GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Error: {err:.1%} con {GS.best_params_}')

Error: 30.1% con {'logreg__C': 0.1, 'logreg__solver': 'liblinear', 'logreg__tol': 1e-05, 'pca__n_components': 0.99, 'poly__degree': 1}
10.5 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [11]:
%%timeit -n1 -r1
scaler = StandardScaler(); pca = PCA(); logreg = LogisticRegression(); poly = PolynomialFeatures()
pipe = Pipeline(steps=[("scaler", scaler), ("poly", poly), ("pca", pca), ("logreg", logreg)])
G = {"pca__n_components": [.9, .99, None],
     "poly__degree": [2],
     "logreg__solver": ["lbfgs", "liblinear"],
     "logreg__tol": [1e-5, 1e-4, 1e-3],
     "logreg__C": [.1, 1, 10]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
err = 1.0 - GS.fit(X_train, y_train).score(X_test, y_test)
print(f'Error: {err:.1%} con {GS.best_params_}')

Error: 28.7% con {'logreg__C': 0.1, 'logreg__solver': 'lbfgs', 'logreg__tol': 1e-05, 'pca__n_components': None, 'poly__degree': 2}
4min 1s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


El resultado con características cuadráticas es mejor que con lineales

Probaremos ahora con cúbicas pero solo con una pequeña porción de los datos

In [13]:
%%timeit -n1 -r1
scaler = StandardScaler(); pca = PCA(); logreg = LogisticRegression(); poly = PolynomialFeatures()
pipe = Pipeline(steps=[("scaler", scaler), ("poly", poly), ("pca", pca), ("logreg", logreg)])
G = {"pca__n_components": [.9, .99, None],
     "poly__degree": [3],
     "logreg__solver": ["lbfgs", "liblinear"],
     "logreg__tol": [1e-5, 1e-4, 1e-3],
     "logreg__C": [.1, 1, 10]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
err = 1.0 - GS.fit(X_train[:100, :], y_train[:100]).score(X_test, y_test)
print(f'Error: {err:.1%} con {GS.best_params_}')

Error: 41.4% con {'logreg__C': 0.1, 'logreg__solver': 'lbfgs', 'logreg__tol': 1e-05, 'pca__n_components': 0.9, 'poly__degree': 3}
12.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


El error a priori es mayor, y por lo tanto el ajuste peor. Probaremos con más datos para comprobar el ajuste

In [14]:
%%timeit -n1 -r1
scaler = StandardScaler(); pca = PCA(); logreg = LogisticRegression(); poly = PolynomialFeatures()
pipe = Pipeline(steps=[("scaler", scaler), ("poly", poly), ("pca", pca), ("logreg", logreg)])
G = {"pca__n_components": [.9, .99, None],
     "poly__degree": [3],
     "logreg__solver": ["lbfgs", "liblinear"],
     "logreg__tol": [1e-5, 1e-4, 1e-3],
     "logreg__C": [.1, 1, 10]}
GS = GridSearchCV(pipe, G, scoring='accuracy', refit=True, cv=5)
err = 1.0 - GS.fit(X_train[:500, :], y_train[:500]).score(X_test, y_test)
print(f'Error: {err:.1%} con {GS.best_params_}')

Error: 36.7% con {'logreg__C': 0.1, 'logreg__solver': 'liblinear', 'logreg__tol': 1e-05, 'pca__n_components': None, 'poly__degree': 3}
1min 37s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


El error mejora al usar más datos, así que sería recomendable utilizar porciones mayores de los datos en caso de que el tiempo que consuma sea factible